### Introducción
El presente proyecto se centra en el desarrollo de un sistema de recomendación de películas basado en contenido. A través del análisis de datos de películas obtenidos de la API de TMDB, este sistema busca predecir películas similares basándose en las preferencias del usuario, ofreciendo recomendaciones personalizadas que mejoran la experiencia de exploración de contenidos cinematográficos.

El proceso se inicia con la carga y preparación de dos conjuntos de datos esenciales: detalles de películas y créditos de películas, almacenados respectivamente en tmdb_5000_movies.csv y tmdb_5000_credits.csv. Posteriormente, se realiza la integración de estos conjuntos en una única estructura de datos, enfocándose en características relevantes como el título, géneros, palabras clave, elenco y equipo de producción. Esta fusión permite una comprensión más amplia de cada película, fundamentando las bases para un análisis más profundo.

La transformación de los datos juega un papel crucial en el proyecto. Se procesan las columnas con formato JSON para extraer información significativa, como los nombres de los géneros, palabras clave, los tres principales actores y el director, limpiando y consolidando estos datos en una columna de etiquetas (tags) que resumen los aspectos más destacados de cada película. Además, se implementa una estrategia de preprocesamiento de texto para eliminar las distancias entre las palabras, optimizando así la precisión del modelo.

La columna de etiquetas se vectoriza utilizando el método TF-IDF, convirtiendo el texto en una matriz numérica que refleja la importancia de las palabras dentro del conjunto de datos. Esta vectorización es fundamental para el cálculo de similitudes entre películas, utilizando la similitud del coseno para identificar las películas más parecidas a una dada, basándose en sus características.

El modelo de recomendación se construye sobre el algoritmo k-nearest neighbors (KNN), aprovechando la matriz de similitud del coseno para sugerir las cinco películas más similares a cualquier película seleccionada. Este enfoque permite una recomendación precisa y personalizada, mejorando significativamente la experiencia del usuario en la exploración de nuevos contenidos cinematográficos.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np
import os
import joblib
import sqlite3
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import load_npz, save_npz
from joblib import dump

In [2]:
url_1 = "https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv"
df_1 = pd.read_csv(url_1)

url_2 = "https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv"
df_2 = pd.read_csv(url_2)


In [3]:
# Imprimir los nombres de las columnas de cada DataFrame
print("Columnas en df_1 (movies):", df_1.columns.tolist())
print("Columnas en df_2 (credits):", df_2.columns.tolist())

Columnas en df_1 (movies): ['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count']
Columnas en df_2 (credits): ['movie_id', 'title', 'cast', 'crew']


In [4]:
# Establecer la ruta al archivo de base de datos
database_path = '../data/raw/my_database.db'

# Conectar a la base de datos SQLite, creando el archivo en la ubicación especificada
conn = sqlite3.connect(database_path)

# Guardar df_1 y df_2 en la base de datos como tablas separadas
df_1.to_sql('movies', conn, index=False, if_exists='replace')
df_2.to_sql('credits', conn, index=False, if_exists='replace')

# Realizar la unión SQL para crear una tercera tabla con la información unificada
query = """
SELECT m.id AS movie_id, m.title, m.overview, m.genres, m.keywords, c.cast, c.crew
FROM movies m
JOIN credits c ON m.title = c.title
"""

# Ejecutar la consulta y cargar el resultado en un nuevo DataFrame
df = pd.read_sql(query, conn)

# Mostrar las primeras filas para verificar
print(df.head())

# Cerrar la conexión
conn.close()

   movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  In the 22nd century, a paraplegic Marine is di...   
1  Captain Barbossa, long believed to be dead, ha...   
2  A cryptic message from Bond’s past sends him o...   
3  Following the death of District Attorney Harve...   
4  John Carter is a war-weary, former military ca...   

                                              genres  \
0  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  [{"id": 28, "name": "Action"}, {"id":

In [5]:
# Imprimir las primeras entradas de las columnas de JSON para inspeccionarlas
print("Genres sample:")
print(df['genres'].head())

print("\nKeywords sample:")
print(df['keywords'].head())

print("\nCast sample:")
print(df['cast'].head())

print("\nCrew sample:")
print(df['crew'].head())


Genres sample:
0    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1    [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3    [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4    [{"id": 28, "name": "Action"}, {"id": 12, "nam...
Name: genres, dtype: object

Keywords sample:
0    [{"id": 1463, "name": "culture clash"}, {"id":...
1    [{"id": 270, "name": "ocean"}, {"id": 726, "na...
2    [{"id": 470, "name": "spy"}, {"id": 818, "name...
3    [{"id": 849, "name": "dc comics"}, {"id": 853,...
4    [{"id": 818, "name": "based on novel"}, {"id":...
Name: keywords, dtype: object

Cast sample:
0    [{"cast_id": 242, "character": "Jake Sully", "...
1    [{"cast_id": 4, "character": "Captain Jack Spa...
2    [{"cast_id": 1, "character": "James Bond", "cr...
3    [{"cast_id": 2, "character": "Bruce Wayne / Ba...
4    [{"cast_id": 5, "character": "John Carter", "c...
Name: cast, dtype: object

Crew sample:
0    [{"credit_id": "52fe48009

In [6]:
# Función para extraer los nombres de los keywords
def extract_keyword_names(keywords_json):
    try:
        keywords = json.loads(keywords_json)
        return [keyword['name'] for keyword in keywords]
    except json.JSONDecodeError:
        return []

# Función para extraer los nombres de los tres primeros actores del cast
def extract_cast_names(cast_json):
    try:
        cast = json.loads(cast_json)
        return [member['name'] for member in cast[:3]]
    except json.JSONDecodeError:
        return []

# Función para encontrar el nombre del director en el crew
def find_director(crew_json):
    try:
        crew = json.loads(crew_json)
        directors = [member['name'] for member in crew if member['job'] == 'Director']
        return directors[0] if directors else None
    except json.JSONDecodeError:
        return None

In [7]:
# Prueba de las funciones con las primeras filas de las columnas
print("Keywords processed sample:")
print(df['keywords'].head().apply(extract_keyword_names))

print("\nCast processed sample:")
print(df['cast'].head().apply(extract_cast_names))

print("\nCrew processed sample:")
print(df['crew'].head().apply(find_director))

Keywords processed sample:
0    [culture clash, future, space war, space colon...
1    [ocean, drug abuse, exotic island, east india ...
2    [spy, based on novel, secret agent, sequel, mi...
3    [dc comics, crime fighter, terrorist, secret i...
4    [based on novel, mars, medallion, space travel...
Name: keywords, dtype: object

Cast processed sample:
0    [Sam Worthington, Zoe Saldana, Sigourney Weaver]
1       [Johnny Depp, Orlando Bloom, Keira Knightley]
2        [Daniel Craig, Christoph Waltz, Léa Seydoux]
3        [Christian Bale, Michael Caine, Gary Oldman]
4      [Taylor Kitsch, Lynn Collins, Samantha Morton]
Name: cast, dtype: object

Crew processed sample:
0        James Cameron
1       Gore Verbinski
2           Sam Mendes
3    Christopher Nolan
4       Andrew Stanton
Name: crew, dtype: object


In [8]:
# Aplicar las funciones a las columnas correspondientes en el DataFrame completo
df['keywords'] = df['keywords'].apply(extract_keyword_names)
df['cast'] = df['cast'].apply(extract_cast_names)
df['crew'] = df['crew'].apply(find_director)

In [9]:
# Para cada columna en ['genres', 'keywords', 'cast', 'crew'], 
# elimina los espacios para evitar confusiones en el modelo de recomendación
def remove_spaces(l):
    return [i.replace(' ', '') for i in l]

df['genres'] = df['genres'].apply(remove_spaces)
df['keywords'] = df['keywords'].apply(remove_spaces)
df['cast'] = df['cast'].apply(remove_spaces)
df['crew'] = df['crew'].apply(lambda x: x.replace(' ', '') if x else '')

In [10]:
# Crear la columna 'tags' combinando 'overview' (convertida a lista de palabras si no es None),
# 'genres', 'keywords', 'cast', y 'crew' en una cadena de texto única.
df['tags'] = df.apply(lambda row: ' '.join(
    (row['overview'] or '').split() +     # Para no tener error cuando 'overview' esté vacía 
    row['genres'] +
    row['keywords'] +
    row['cast'] +
    ([row['crew']] if row['crew'] else [])
), axis=1)

# Muestra una muestra de la columna 'tags' para verificar
print(df['tags'].head())

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: tags, dtype: object


In [11]:
df.columns

Index(['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew',
       'tags'],
      dtype='object')

In [12]:
df['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. [ { " i d " :  2 8 ,  " n a m e " :  " A c t i o n " } ,  { " i d " :  1 2 ,  " n a m e " :  " A d v e n t u r e " } ,  { " i d " :  1 4 ,  " n a m e " :  " F a n t a s y " } ,  { " i d " :  8 7 8 ,  " n a m e " :  " S c i e n c e  F i c t i o n " } ] cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [13]:
# Guardar el DataFrame como CSV
file_path = '../data/interim/df.csv'
df.to_csv(file_path, index=False)

In [14]:
# Transformamos 'tags' en una matriz de vectores numéricos
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['tags'])

In [15]:
# Guardar la matriz como TF-IDF
file_path_matrix = '../data/processed/tfidf_matrix.npz'
save_npz(file_path_matrix, tfidf_matrix)

In [16]:
# Calcular la similitud del coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [17]:
# Función para recomendar películas similares
def recommend(movie_title, cosine_sim=cosine_sim, df=df):
    # Manejar posibles errores si el título no se encuentra
    if movie_title not in df['title'].values:
        return f"Movie title '{movie_title}' not found in the dataset."
    
    # Obtener el índice de la película que coincide con el título
    idx = df.loc[df['title'] == movie_title].index[0]
    
    # Obtener las puntuaciones de similitud para esa película con todas las demás
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Ordenar las películas en función de las puntuaciones de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtener los índices de las 'k' películas más similares (excluyendo la propia película)
    sim_scores = sim_scores[1:6]  # Aquí 'k' es 5
    
    # Obtener los índices de las películas
    movie_indices = [i[0] for i in sim_scores]
    
    # Devolver los títulos de las películas más similares
    return df['title'].iloc[movie_indices]

In [18]:
# Probamos la función
recommendations = recommend("American Beauty")
print(recommendations)

2826             Chronicle
3988         Hustle & Flow
1082    Revolutionary Road
1750    Notes on a Scandal
1444     The Shipping News
Name: title, dtype: object


In [19]:
# Guardar el vectorizador TF-IDF
dump(tfidf, '../models/tfidf_vectorizer.joblib')

['../models/tfidf_vectorizer.joblib']

Conclusión Final
El proyecto de sistema de recomendación de películas ha demostrado ser una iniciativa exitosa en la aplicación de técnicas avanzadas de procesamiento de lenguaje natural y aprendizaje automático para ofrecer recomendaciones personalizadas. A través de un cuidadoso proceso de limpieza, preparación y análisis de datos, se ha desarrollado un sistema capaz de entender las preferencias del usuario y sugerir películas similares que probablemente sean de su interés.

La integración y transformación de datos provenientes de diversas fuentes han sido esenciales para construir una base sólida para el análisis. La implementación de vectorización TF-IDF y la similitud del coseno han permitido cuantificar la relación entre películas de manera eficaz, destacando la importancia de las técnicas de NLP en el campo del análisis de recomendaciones.

El uso del algoritmo KNN como base para el modelo de recomendación ha demostrado ser eficaz en la identificación de películas similares, ofreciendo a los usuarios una selección personalizada y relevante. 

En conclusión, este proyecto ha alcanzado su objetivo de desarrollar un sistema de recomendación de películas basado en contenido, estableciendo un modelo sólido que puede ser extendido o mejorado con datos adicionales, técnicas de procesamiento de texto más avanzadas o incluso integrando otros modelos predictivos. 